In [ ]:
import csv
import pymongo

In [ ]:
MONGO_DB = "bsocoverage"
MONGO_COLLECTION = "publications"

In [ ]:
mongo_database = pymongo.MongoClient()[MONGO_DB]
mongo_collection = mongo_database[MONGO_COLLECTION]

In [ ]:
# Set year with priority to FOSM
actions = []
for publication in mongo_collection.find():
  if publication.get("year_fosm"):
    actions.append(pymongo.UpdateOne({ "id": publication.get("id") }, { "$set": { "year": publication.get("year_fosm") } }, upsert=True))
  elif publication.get("year_openalex"):
    actions.append(pymongo.UpdateOne({ "id": publication.get("id") }, { "$set": { "year": publication.get("year_openalex") } }, upsert=True))
if len(actions) > 0:
  mongo_collection.bulk_write(actions, ordered=False)

In [ ]:
# List errors about publication year consistency accross FOSM and OpenAlex
with open("output/errors_publication_year.csv", "w", encoding="UTF8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "year_fosm", "year_openalex"])
    for publication in mongo_collection.find():
        if publication.get("year_fosm") and publication.get("year_openalex") and publication.get("year_fosm") != publication.get("year_openalex"):
            writer.writerow([publication.get('id'), publication.get('year_fosm'), publication.get('year_openalex')])

In [ ]:
# List DOI from 2021 existing in OpenAlex but missing in FOSM
results = mongo_collection.find({ "is_in_openalex": True, "is_in_fosm": { "$exists": False }, "year": 2021 }).limit(200)
with open("output/doi_of_publications_from_openalex_missing_in_fosm_from_2021.csv", "w", encoding="UTF8") as f:
  writer = csv.writer(f)
  writer.writerow(["doi"])
  for result in results:
    writer.writerow([result.get("id")])